##### Copyright 2019 The TensorFlow Authors.

# Convolutional Neural Network (CNN)

### Import TensorFlow

In [ ]:
import os
import tensorflow as tf

from tensorflow.keras import datasets, layers, models, regularizers
import matplotlib.pyplot as plt

import numpy as np

# custom made functions for this Machine Learning dataset
import customUtils as cu

# used for importing/exporting matlab data
from scipy.io import savemat
from scipy.io import loadmat

### disable GPU in case of errors

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

## setup gpu if present

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

### Preparing the dataset

In [ ]:
#import train data and correctly format images and labels for tensorflow
X, y_vec = cu.importImageTrainDataFromMatlab('dataset.mat', data_size=6500)
X = X / 255.0
train_images = np.reshape(X,(X.shape[0], 50, 50, 3))
y = np.zeros(y_vec.shape[0])
for j in range(y.shape[0]):
    y[j] = np.where(y_vec[j,:] == 1)[0][0]
train_labels = y.reshape(y.shape[0],1).astype(int)

X_test, y_test_vec = cu.importImageTestDataFromMatlab('dataset.mat', data_size=1625)
X_test = X_test / 255.0
test_images = np.reshape(X_test,(X_test.shape[0], 50, 50, 3))
y_test = np.zeros(y_test_vec.shape[0])
for j in range(y_test.shape[0]):
    y_test[j] = np.where(y_test_vec[j,:] == 1)[0][0]
test_labels = y_test.reshape(y_test.shape[0],1).astype(int)

### Verify the data

To verify that the dataset looks correct, let's plot the first image from every PCB class from the training set and display the class name below each image:


In [ ]:
class_names = ['Arduino Mega 2560 (Blue)', 'Arduino Mega 2560 (Black)', 'Arduino Mega 2560 (Black and Yellow)',
               'Arduino Due','Beaglebone Black', 'Arduino Uno (Green)', 'Raspberry Pi 3 B+', 'Raspberry Pi 1 B+',
               'Arduino Uno Camera Shield', 'Arduino Uno (Black)', 'Arduino Uno WiFi Shield', 'Arduino Leonardo', 'Raspberry Pi A+']

plt.figure(figsize=(22,5))
for i in range(13):
    plt.subplot(2,7,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i*500])
    plt.xlabel(class_names[train_labels[i*500][0]])
plt.show()

### Create the convolutional base

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to these dimensions, color_channels refers to (R,G,B). In this example, you will configure your CNN to process inputs of shape (32, 32, 3), which is the format of CIFAR images. You can do this by passing the argument `input_shape` to your first layer.


In [ ]:
HLS=10
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# Dense part of the NN
model.add(layers.Flatten())
model.add(layers.Dense(HLS, activation='sigmoid'))
model.add(layers.Dense(13))

Here's the complete architecture of your model:

In [ ]:
model.summary()

### Compile and train the model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, epochs=5, 
                    validation_data=(test_images, test_labels))

### Evaluate the model

In [ ]:
plt.plot([elem*100 for elem in history.history['accuracy']], label='train accuracy', marker='o')
plt.plot([elem*100 for elem in history.history['val_accuracy']], label = 'val accuracy', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([50, 100])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)

In [ ]:
model.save('CNNmodels/CNNmodelHLS' + str(HLS))